In [26]:
import os
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cufflinks as cf

data_dir = Path('../data')

income_groups_1639 = pd.read_csv(data_dir / 'interim' / 'income_groups_1639.csv', index_col='district')
income_1880 = pd.read_csv(data_dir / 'interim' / 'income_tax_record_1880.csv', index_col=0)
income_1880['total_tax'] = income_1880.sum(axis=1)

/tmp/ipykernel_28196/2929370045.py:13: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



In [27]:
totals = income_groups_1639.sum(axis=0)
percentiles = (totals / totals.sum()).round(2)
cum_percentiles = percentiles[::-1].cumsum()
cum_percentiles

no_tax    0.08
12_ore    0.73
16_ore    1.00
dtype: float64

In [28]:
breaks = income_1880.total_tax.quantile(cum_percentiles)
breaks

0.08      1.0
0.73      3.0
1.00    562.0
Name: total_tax, dtype: float64

In [39]:
classes = pd.cut(
    income_1880.total_tax,
    bins=[0] + list(breaks),
    labels=[1, 2, 3],
)
classes

0       2
1       3
2       1
3       1
4       3
       ..
3331    1
3332    1
3333    1
3334    1
3335    1
Name: total_tax, Length: 3336, dtype: category
Categories (3, int64): [1 < 2 < 3]